- Setup a Langchain project and get credentials

In [1]:
import dotenv

# OPENAI_API_KEY
# LANGFUSE_SECRET_KEY
# LANGFUSE_PUBLIC_KEY
# LANGFUSE_HOST="https://cloud.langfuse.com"

dotenv.load_dotenv('./env.txt')

True

# Playground demo

- playground lets us experiment with different prompts based on our use cases
- once a prompt gives stable output we can then store it for future use


In [41]:
message = """
Answer this question using the provided context only.

{question}

Context:
{context}
"""

# Prompt management demo
- use can then version and use these prompts in production
- prompts are fetch in real-time and therefore can be used to quicky change the application behaviour

# Decorator based integration

In [42]:
from langfuse.decorators import observe, langfuse_context
import openai
 

# decorator to monitor trace
@observe(as_type="generation")
def call_openai(session_id, user_id, **kwargs):

  # add metadata to the context
  langfuse_context.update_current_observation(
      input=kwargs["messages"],
      model=kwargs["model"],
      metadata={"max_tokens": kwargs["max_tokens"], "type": "testing"}
  )

  # optional arguments
  langfuse_context.update_current_trace(
    session_id=session_id, 
    user_id=user_id, 
    metadata={"is_openai": bool("gpt" in kwargs["model"])}
)
 
  # return value also monitored
  return openai.chat.completions.create(**kwargs).choices[0].message.content
 
@observe()
def main(session_id = "", user_id = ""):
  return call_openai(
    session_id,
    user_id,
      model="gpt-4o-mini",
      max_tokens=200,
      messages=[
          {"role": "user", "content": "What is the meaning of life ?"}
      ],
      
  )
 

In [43]:
main()

'The meaning of life is a philosophical question that has been debated for centuries. Different individuals and cultures have different beliefs and perspectives on the meaning of life. Some believe that the meaning of life is to seek happiness and fulfillment, others believe it is to serve others and make a positive impact on the world, while some believe that the meaning of life is defined by religious beliefs and principles. Ultimately, the meaning of life is a subjective and personal belief that each individual must discover for themselves.'

In [34]:
main(session_id="testing")

'The meaning of life is a philosophical question that has intrigued humans for centuries. Different people and cultures have different beliefs about the purpose and meaning of life. Some believe it is to seek happiness and fulfillment, others believe it is to follow a certain religious or spiritual path, while others believe it is simply to exist and experience the world around us. Ultimately, the meaning of life is subjective and can be interpreted in many different ways.'

In [35]:
main(session_id="testing", user_id="user_123")

'The meaning of life is a philosophical and existential question that has been debated for centuries. It is a question that has no definitive answer and can vary depending on individual beliefs, values, and perspectives. Some people believe that the meaning of life is to seek happiness and fulfillment, others see it as finding purpose and meaning through personal or spiritual growth, while some believe that life has no inherent meaning and it is up to each individual to create their own meaning. Ultimately, the meaning of life is a subjective and deeply personal question that each individual must grapple with and find their own answer to.'